In [ ]:
import os
import pandas as pd

In [ ]:
data_dir = "./data"
example_submission_path = os.path.join(data_dir, "gender_submission.csv")
train_csv_path = os.path.join(data_dir, "train.csv")
test_csv_path = os.path.join(data_dir, "test.csv")

# Data description

The data has been split into two groups:

- training set (train.csv)
- test set (test.csv)

The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

## Data Dictionary

| Variable | Definition                                 | Key                                            |
|----------|--------------------------------------------|------------------------------------------------|
| survival | Survival                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| sex      | Sex                                        |                                                |
| Age      | Age in years                               |                                                |
| sibsp    | # of siblings / spouses aboard the Titanic |                                                |
| parch    | # of parents / children aboard the Titanic |                                                |
| ticket   | Ticket number                              |                                                |
| fare     | Passenger fare                             |                                                |
| cabin    | Cabin number                               |                                                |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |


## Variable Notes

**pclass**: A proxy for socio-economic status (SES)
- 1st = Upper
- 2nd = Middle
- 3rd = Lower

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...
- Sibling = brother, sister, stepbrother, stepsister
- Spouse = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...
- Parent = mother, father
- Child = daughter, son, stepdaughter, stepson
- Some children travelled only with a nanny, therefore parch=0 for them.

In [ ]:
train_df = pd.read_csv(train_csv_path)

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
def make_ticket(prefix, number):
    return {'ticket_prefix': prefix, 'ticket_number': number}

def transform_ticket_number(ticket_no, use_nan_map=False):
    # TODO: maybe we should make "is_line" category column
    ticket_map = {
        'LINE': -1
    }
    return int(ticket_map.get(ticket_no, ticket_no))

def transform_ticket_prefix(ticket_prefix):
    if ticket_prefix == 'STON/O2.':
        return 'STON/O 2.'
    return ticket_prefix
    
def split_ticket(ticket):
    parts = ticket.split()
    if len(parts) == 0:
        return make_ticket(None, np.pd.nan)
    elif len(parts) == 1:
        return make_ticket(None, transform_ticket_number(parts[0]))
    return make_ticket(
        transform_ticket_prefix(" ".join(parts[0:-1])),
        transform_ticket_number(parts[-1])
    )

In [ ]:
def process_df(original_df):
    ticket_split_df = pd.DataFrame(original_df['Ticket'].apply(split_ticket).tolist())
    return original_df.join(ticket_split_df)

In [ ]:
processed_df = process_df(train_df)
assert(len(processed_df[processed_df['ticket_number'] == pd.np.nan]) == 0)
processed_df

In [ ]:
def prepare_feature_df(processed_df):
    """Turn a processed DataFrame into a DataFrame suitable for an ML model."""
    return processed_df.drop(
        columns=[
            'Ticket',
            'Name',
            # We should try adding this back in after we've built our baseline model.
            # Look at `set(prepared_df['ticket_prefix'])` to see ticket prefixes that
            # might be similar
            'ticket_prefix',
        ]
    )

In [ ]:
prepared_df = prepare_feature_df(processed_df)
prepared_df